In [ ]:
import MDAnalysis as mda
from MDAnalysis import transformations
from MDAnalysis.analysis.rdf import InterRDF, InterRDF_s
from solvation_analysis.solution import Solution
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
import nglview

def visualize(selection):
    mda_view = nglview.show_mdanalysis(selection)
    return mda_view.display()


In [ ]:
# instantiate solution
from solvation_analysis.solution import Solution

In [ ]:
u1 = mda.Universe('output1/topology.pdb',"output1/trajectory.dcd")
u0 = mda.Universe('output1/topology.pdb',"output1/trajectory_equil.dcd")

In [ ]:
# get box volume
# import box size from state.txt
volume_nm1 = pd.read_csv('output1/state.txt')["Box Volume (nm^3)"].values[-1]
# convert to A 10^-8
volume1 = volume_nm1 * 1000
# side of the box
l1 = volume1**(1/3)
# lengths *a*, *b*, *c* are in the MDAnalysis length unit (Å), and angles are in degrees.
box1 = [l1,l1,l1,90,90,90]
# telling trajectory box size
set_dim1 = transformations.boxdimensions.set_dimensions(box1)
# manually set the box size
u1.trajectory.add_transformations(set_dim1) 


In [ ]:
from solvation_analysis import selection

In [ ]:
# these queries will not work for all circumstances, but they work here because we have only two species
# water has no C and the trimer has C, that is what we are using to select atoms


trimer_1 = u1.select_atoms('byres element C')
Cl_1 = u1.select_atoms('byres element Cl')

# select both h2o and h3o+
OH_1= u1.select_atoms('not byres element C Cl').select_atoms('byres element O')

h2O_1 = u1.select_atoms('resid 1:3640')

h3O_1 = OH_1-h2O_1

h2O_O_1 = h2O_1.select_atoms('element O')
h2O_H_1 = h2O_1.select_atoms('element H')
h3O_O_1 = h3O_1.select_atoms('element O')
h3O_H_1 = h3O_1.select_atoms('element H')

# C-O in ether and ester

ether_O_1 = trimer_1.select_atoms('smarts COC=C').select_atoms('element O')
ester_O_1 = trimer_1.select_atoms('smarts COCC=C').select_atoms('element O')

# C=O in ketone and ester
ketone_O_1 = trimer_1.select_atoms('smarts CC(=O)C').select_atoms('element O')
ester_2O_1 = trimer_1.select_atoms('smarts C=O').select_atoms('element O')\
- ketone_O_1


# tertiary N in the center
N_1 = trimer_1.select_atoms('smarts [N;D4]').select_atoms('element N')
# N in the side chains
N3_1 = trimer_1.select_atoms('smarts [N;D3]').select_atoms('element N')


oh_O_1 = trimer_1.select_atoms('smarts [OHX2]')
dimethyl_C_1 = trimer_1.select_atoms('smarts [CH3]')


In [ ]:
visualize(trimer_1+h2O_1+h3O_1+Cl_1)

### If the system is well-equilibrated?

should check more observable quantities to see if the simulation reaches equilibrium

In [ ]:
df1 = pd.read_csv('output1/state.txt')
df1.rename(columns={'#"Step"': 'Step'}, inplace=True)
df1['Step'][-1:]

In [ ]:
# check if RDFs overlap, 
# we need to check different observable quantities to make sure the simulation is well-equilibrated


In [ ]:
rdf = InterRDF(N_1, h2O_O_1)
rdf0_3=rdf.run(start=0,stop = 300)
rdf3_6=rdf.run(start=300,stop = 600)
rdf6_9=rdf.run(start=600,stop = 900)
rdf9_12=rdf.run(start=900,stop = 1200)
rdf12_15=rdf.run(start=1200,stop = 1488)

# https://docs.mdanalysis.org/1.1.0/documentation_pages/analysis/rdf.html
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(rdf0_3.bins,rdf0_3.rdf,"r",label = '0-300')
ax.plot(rdf3_6.bins,rdf3_6.rdf,"b",label = '300-600')
ax.plot(rdf6_9.bins,rdf6_9.rdf,"g",label = '600-900')
ax.plot(rdf9_12.bins,rdf9_12.rdf,"orange",label = '900-1200')
ax.plot(rdf12_15.bins,rdf12_15.rdf,"y",label = '1200-1488')

ax.set_xlabel('Radial Distance (A)')
ax.set_ylabel('Probability Density')
ax.set_title('Tertiary amine N - H2O O RDFs Over Time')

ax.legend()

In [ ]:
rdf = InterRDF(N_1, h3O_O_1)
rdf0_3=rdf.run(start=0,stop = 300)
rdf3_6=rdf.run(start=300,stop = 600)
rdf6_9=rdf.run(start=600,stop = 900)
rdf9_12=rdf.run(start=900,stop = 1200)
rdf12_15=rdf.run(start=1200,stop = 1488)

# https://docs.mdanalysis.org/1.1.0/documentation_pages/analysis/rdf.html
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(rdf0_3.bins,rdf0_3.rdf,"r",label = '0-300')
ax.plot(rdf3_6.bins,rdf3_6.rdf,"b",label = '300-600')
ax.plot(rdf6_9.bins,rdf6_9.rdf,"g",label = '600-900')
ax.plot(rdf9_12.bins,rdf9_12.rdf,"orange",label = '900-1200')
ax.plot(rdf12_15.bins,rdf12_15.rdf,"y",label = '1200-1500')

ax.set_xlabel('Radial Distance (A)')
ax.set_ylabel('Probability Density')
ax.set_title('Tertiary amine N - H3O O RDFs Over Time')

ax.legend()

In [ ]:
rdf = InterRDF(N_1, Cl_1)
rdf0_3=rdf.run(start=0,stop = 300)
rdf3_6=rdf.run(start=300,stop = 600)
rdf6_9=rdf.run(start=600,stop = 900)
rdf9_12=rdf.run(start=900,stop = 1200)
rdf12_15=rdf.run(start=1200,stop = 1488)

# https://docs.mdanalysis.org/1.1.0/documentation_pages/analysis/rdf.html
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(rdf0_3.bins,rdf0_3.rdf,"r",label = '0-300')
ax.plot(rdf3_6.bins,rdf3_6.rdf,"b",label = '300-600')
ax.plot(rdf6_9.bins,rdf6_9.rdf,"g",label = '600-900')
ax.plot(rdf9_12.bins,rdf9_12.rdf,"orange",label = '900-1200')
ax.plot(rdf12_15.bins,rdf12_15.rdf,"y",label = '1200-1500')

ax.set_xlabel('Radial Distance (A)')
ax.set_ylabel('Probability Density')
ax.set_title('Tertiary amine N - Cl RDFs Over Time')

ax.legend()

len(u1.trajectory)


In [ ]:
plt.subplots(2,1,figsize = (12,10))
ax0_1 = plt.subplot(2,1,1)

ax0_1.plot(df1['Step'][:2000],df1['Density (g/mL)'][:2000])
plt.ylim([1.1,1.15])
ax0_1.set_xlabel("Step")
ax0_1.set_ylabel("Density")
ax0_1.set_title("first 2 ns")

ax0_2 = plt.subplot(2,1,2)

ax0_2.plot(df1['Step'],df1['Density (g/mL)'])
plt.ylim([1.1,1.15])
ax0_2.set_xlabel("Step")
ax0_2.set_ylabel("Density")
ax0_2.set_title("total time")


In [ ]:
# moving average
def moving_average(a, n=5) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

mva_dens = moving_average(list(df1['Density (g/mL)']))


In [ ]:
plt.subplots(2,1,figsize = (12,10))
ax0_1 = plt.subplot(2,1,1)

ax0_1.plot(df1['Step'][:2000],mva_dens[:2000])
plt.ylim([1.1,1.15])
ax0_1.set_xlabel("Step")
ax0_1.set_ylabel("Density")
ax0_1.set_title("first 2 ns")

ax0_2 = plt.subplot(2,1,2)

ax0_2.plot(df1['Step'][:-4],mva_dens)
plt.ylim([1.1,1.15])
ax0_2.set_xlabel("Step")
ax0_2.set_ylabel("Density")
ax0_2.set_title("total time")


In [ ]:
# equilibration summary
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(10,10))
fig.suptitle("Equilibration Summary",  fontsize=16)

ax1.plot(df1['Step'][:5000],df1['Potential Energy (kJ/mole)'][:5000])
ax1.set_ylabel('Potential Energy (kJ/mole)')

ax2.plot(df1['Step'][:5000],df1['Temperature (K)'][:5000],'violet')
ax2.set_ylabel('Temperature (K)')


ax3.plot(df1['Step'][:5000],df1['Box Volume (nm^3)'][:5000],'grey')
ax3.set_xlabel('timestep')
ax3.set_ylabel('Box Volume (nm^3)')
ax3.set_ylim(top = 260, bottom = 220)


ax4.plot(df1['Step'][:5000],df1['Density (g/mL)'][:5000],'orange')
ax4.set_xlabel('timestep')
ax4.set_ylabel('Density (g/mL)')
ax4.set_ylim(top = 1.150, bottom = 1.100)

## Solvation Analysis- Coordination Number

### Tertiary amine N

In [ ]:
solution1_3 = Solution(N_1, {'H2O': h2O_1, 'H3O': h3O_1, 'Cl': Cl_1})

In [ ]:
print(solution1_3.radii)

In [ ]:
solution1_3.run()

In [ ]:
solution1_3.radii['H3O']=4.3

In [ ]:
# check that our new radius looks good
solution1_3.plot_solvation_radius('H2O')
solution1_3.plot_solvation_radius('H3O')
solution1_3.plot_solvation_radius('Cl')
plt.show()

In [ ]:
# inspect the radii
print(solution1_3.radii)
solution1_3.run()

In [ ]:
solution1_3.coordination.cn_dict

In [ ]:
solution1_3.coordination.cn_by_frame

In [ ]:
num3 = len(N_1)/50
cn_byframe3 = solution1_3.coordination.cn_by_frame/num3
cn_byframe3

In [ ]:
# Coordination Number Over Time
plt.figure(figsize=(10, 5))
plt.plot(cn_byframe3.iloc[1]/num3,label='Tertiary N- H2O')
plt.legend()
plt.title('Coordination Number Over Time')
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(cn_byframe3.iloc[0],label='Tertiary N- Cl-')
plt.legend()
plt.title('Coordination Number Over Time')
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(cn_byframe3.iloc[2],label='Tertiary N- H3O+')
plt.legend()
plt.title('Coordination Number Over Time')
plt.show()

In [ ]:
print("Cl-:",sum(cn_byframe3.iloc[0]))
print("H2O:", sum(cn_byframe3.iloc[1]))
print("H3O+:",sum(cn_byframe3.iloc[2]))

In [ ]:
cn1_3 = {}
for i in solution1_3.coordination.cn_dict:
    cn1_3[i]= solution1_3.coordination.cn_dict[i]/num3
cn1_3

### Secondary Amine N

In [ ]:
solution1_6 = Solution(N3_1, {'H2O': h2O_1, 'H3O': h3O_1, 'Cl': Cl_1})

In [ ]:
solution1_6.run()

In [ ]:
solution1_6.plot_solvation_radius('H2O')
solution1_6.plot_solvation_radius('H3O')
solution1_6.plot_solvation_radius('Cl')
plt.show()

In [ ]:
solution1_6.radii['H3O']= 3.7
solution1_6.radii['Cl']= 4
solution1_6.plot_solvation_radius('H2O')
solution1_6.plot_solvation_radius('H3O')
solution1_6.plot_solvation_radius('Cl')
plt.show()

In [ ]:
solution1_6.run()

In [ ]:
solution1_6.coordination.cn_dict

In [ ]:
num6 = len(N3_1)/50
cn1_6 = {}
for i in solution1_6.coordination.cn_dict:
    cn1_6[i]= solution1_6.coordination.cn_dict[i]/num6
cn1_6

### summary

In [ ]:
cn = {"N_center": cn1_3,
      "N_side": cn1_6}
import pandas as pd
CN = pd.DataFrame(cn)
CN

## Visualization

In [ ]:
visualize(h2O_1+h3O_1 + trimer_1+ Cl_1)

In [ ]:
visualize(trimer_1+ Cl_1)

In [ ]:
rdf1_3 = InterRDF(N_1, h2O_O_1)
rdf1_6 = InterRDF(N3_1, h2O_O_1)
rdf1_3 = InterRDF(N_1, h2O_O_1)
rdf1_6 = InterRDF(N3_1, h2O_O_1)
rdf3_3 = InterRDF(N_1, Cl_1)
rdf3_6 = InterRDF(N3_1, Cl_1)

rdf1_3.run()
rdf1_6.run()
rdf2_3.run()
rdf2_6.run()
rdf3_3.run()
rdf3_6.run()

In [ ]:

fig1,(ax1, ax2, ax3) = plt.subplots(1,3,sharey = 'row', figsize =(18,5))
fig1.suptitle('Comparing Secondary and Tertiary Amine N', fontsize = 15)

ax1.plot(rdf1_6.bins, rdf1_6.rdf, "r-",label = "Secondary amine N")
ax1.plot(rdf1_3.bins, rdf1_3.rdf, "b-",label = "Tertiary amine N")

ax1.set_xlabel("Radial Distance (A)", fontsize = 13)
ax1.set_ylabel("Probability Density of H2O", fontsize = 13)
ax1.legend( fontsize = 13)


ax2.plot(rdf2_6.bins, rdf2_6.rdf, "r-",label = "Secondary amine N")
ax2.plot(rdf2_3.bins, rdf2_3.rdf, "b-",label = "Tertiary amine N")

ax2.set_xlabel("Radial Distance (A)", fontsize = 13)
ax2.set_ylabel("Probability Density of H3O+", fontsize = 13)
ax2.legend(fontsize = 13)


ax3.plot(rdf3_6.bins, rdf3_6.rdf, "r-",label = "Secondary amine N")
ax3.plot(rdf3_3.bins, rdf3_3.rdf, "b-",label = "Tertiary amine N")

ax3.set_xlabel("Radial Distance (A)", fontsize = 13)
ax3.set_ylabel("Probability Density of Cl-", fontsize = 13)
ax3.legend(fontsize = 13)



In [ ]:
fig2, ((ax4,ax5)) = plt.subplots(1,2,sharex='col', sharey='row', figsize=(12,5))

ax4.plot(rdf1_3.bins, rdf1_3.rdf, "r-",label = "H2O")
ax4.plot(rdf2_3.bins, rdf2_3.rdf, "b-",label = "H3O+")
ax4.plot(rdf3_3.bins, rdf3_3.rdf, "g-",label = "Cl-")


ax4.set_xlabel("Radial Distance (A)",fontsize = 13)
ax4.set_ylabel("Probability Density",fontsize = 13)
ax4.set_title('Tertiary amine N RDFs',fontsize = 13)
ax4.legend(fontsize = 13)



ax5.plot(rdf1_6.bins, rdf1_6.rdf, "r-",label = "H2O")
ax5.plot(rdf2_6.bins, rdf2_6.rdf, "b-",label = "H3O+")
ax5.plot(rdf3_6.bins, rdf3_6.rdf, "g-",label = "Cl-")


ax5.set_xlabel("Radial Distance (A)",fontsize = 13)
ax5.set_ylabel("Probability Density",fontsize = 13)
ax5.set_title('Secondary amine N RDFs',fontsize = 13)
ax5.legend(fontsize = 13)

In [ ]:
# compare RDFs (shared x and y)

plt.subplots(1, 3, sharex='col', sharey='row',figsize = (35,10))

# fig1
ax1 = plt.subplot(1,3,1)
ax1.plot(rdf1_6.bins, rdf1_6.rdf, "bx-",label = "N_side")
ax1.plot(rdf1_3.bins, rdf1_3.rdf, "y-",label = "N_center")
ax1.set_xlabel("Radial Distance (A)")
ax1.set_ylabel("Probability Density of H2O")
ax1.legend()

# fig2
ax2 = plt.subplot(1,3,2)
ax2.plot(rdf2_6.bins, rdf2_6.rdf, "bx-",label = "N_side")
ax2.plot(rdf2_3.bins, rdf2_3.rdf, "y-",label = "N_center")
ax2.set_xlabel("Radial Distance (A)")
ax2.set_ylabel("Probability Density of H3O+")
ax2.legend()

# fig3
ax3 = plt.subplot(1,3,3)
ax3.plot(rdf3_6.bins, rdf3_6.rdf, "bx-",label = "N_side")
ax3.plot(rdf3_3.bins, rdf3_3.rdf, "y-",label = "N_center")
ax3.set_xlabel("Radial Distance (A)")
ax3.set_ylabel("Probability Density of Cl-")
ax3.legend()


## Find Minima on the RDF

In [ ]:
from solvation_analysis.rdf_parser import plot_interpolation_fit, identify_solvation_cutoff
from scipy.signal import find_peaks


In [ ]:
cutoff1_3 = identify_solvation_cutoff(rdf1_3.bins, rdf1_3.rdf)

In [ ]:
fig, ax = plot_interpolation_fit(rdf1_3.bins, rdf1_3.rdf)
plt.show()